# Fitbit Exploration
For an explanation on the variables, take a look at the [data dictionary created by Fitabase](https://www.fitabase.com/media/1546/fitabasedatadictionary.pdf).

## Sleep Schedule Check
In this notebook we take a look at how participants' sleep schedules might have shifted due to the pandemic. We have to include participants from the first cohort only since these are the ones that would have had data from before the seriousness of the pandemic started to affect students i.e. Spring Break.

In [1]:
import os
import sys
sys.path.append('../')

from src.features import build_features
from src.visualization import visualize
from src.reports import make_report

import pandas as pd
import numpy as np

from datetime import datetime, timedelta

import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.dates as mdates

# Sleep Data
We have sleep data recorded in slightly different ways from the two cohorts.

## BPEACE1
The BPEACE1 data is downloaded directly from [Fitabase](https://app.fitabase.com/DownloadData/Project/0f0d46af-cd33-4254-a73a-4611f951c3c5) and all the information is contained in the ```sleepLogInfo``` data file.

In [6]:
c1 = pd.read_csv("../data/raw/bpeace1/fitbit/sleepLogInfo_merged.csv",
                parse_dates=['StartTime'],infer_datetime_format=True)
c1.head()

,Id,LogId,StartTime,Duration,Efficiency,IsMainSleep,MinutesAfterWakeup,MinutesAsleep,MinutesToFallAsleep,TimeInBed,AwakeCount,AwakeDuration,RestlessCount,RestlessDuration
0,101,25904802628,2020-02-07 22:17:30,31980000,89,True,0,477,0,533,5,12,23,44
1,101,25904802629,2020-02-09 00:14:00,22800000,94,True,0,357,0,380,3,5,13,18
2,101,25904802630,2020-02-09 23:35:00,32340000,89,True,0,479,0,539,4,14,23,46
3,101,25904802631,2020-02-11 01:05:00,23940000,92,True,0,368,0,399,2,4,13,27
4,101,25904802632,2020-02-12 00:15:30,22860000,93,True,0,354,0,381,2,5,15,22


In [8]:
n = len(c1['Id'].unique())
print(f'Number of participants: {n}')

Number of participants: 77


## BPEACE2
We have already processed the BPEACE2 data into something easy to import.

In [7]:
c2 = pd.read_csv("../data/processed/bpeace2-fitbit-sleep-daily.csv",
                 index_col=0,parse_dates=['date','endTime','startTime'],infer_datetime_format=True)
c2.head()

,dateOfSleep,duration,efficiency,endTime,infoCode,isMainSleep,logId,minutesAfterWakeup,minutesAsleep,minutesAwake,minutesToFallAsleep,startTime,timeInBed,beiwe
date,,,,,,,,,,,,,,
2020-05-14,2020-05-14,24360000,97,2020-05-14 07:13:00,0,True,27170628887,0,379,27,0,2020-05-14 00:27:00,406,hfttkth7
2020-05-15,2020-05-15,29580000,87,2020-05-15 08:06:30,0,True,27187368950,8,392,101,0,2020-05-14 23:53:30,493,hfttkth7
2020-05-16,2020-05-16,19740000,95,2020-05-16 04:57:00,0,True,27197675883,7,287,42,0,2020-05-15 23:28:00,329,hfttkth7
2020-05-17,2020-05-17,26820000,96,2020-05-17 09:28:30,0,True,27214680283,8,403,44,0,2020-05-17 02:01:30,447,hfttkth7
2020-05-18,2020-05-18,24960000,92,2020-05-18 07:20:00,0,True,27227113904,0,351,65,0,2020-05-18 00:24:00,416,hfttkth7


## ID Crossover
Now we need a way to link the participants across the the two studies.

### BPEACE1
The ID list includes participant name

In [10]:
c1_id = pd.read_csv("../data/raw/bpeace1/admin/id_list.csv")
c1_id.head()

,Record ID,Beiwe ID,BB No,Actual BB No,Fitbit No,UT EID,First,Last,Email
0,162,cjbncgvw,2.020011e+13,29.0,20200110045001,aso544,Sofia,Ocegueda,NaN
1,18,g46wfzjn,2.020011e+13,28.0,20200110028001,aja3295,Aaron,Alterman,NaN
2,90,4mydzypv,2.020011e+13,27.0,20200110059001,zmm434,Zoe,McDowell,NaN
3,196,mi3pnw3b,2.020011e+13,26.0,20200110044001,etn335,elizabeth,nguyen,NaN
4,49,b9jm44nv,2.020011e+13,25.0,20200110046001,cbj582,Colleen,Jones,NaN


### BPEACE2
This ID list doesn't originally contain the participants, but we can add those.

In [ ]:
c2_id = pd.read_csv("../data/b")